In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk import pos_tag
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
import pickle
import warnings
warnings.filterwarnings("ignore")

In [ ]:
train=pd.read_csv('../input/us-patent-phrase-to-phrase-matching/train.csv')
test=pd.read_csv('../input/us-patent-phrase-to-phrase-matching/test.csv')
train.head()

In [ ]:
train['score'].value_counts().plot(kind='bar')

In [ ]:
train['cate']=train['context'].apply(lambda x: x[0])
test['cate']=test['context'].apply(lambda x: x[0])
train.head()

In [ ]:
dict_cpc = {    'A': 'Human Necessities',
                'B': 'Operations and Transport',
                'C': 'Chemistry and Metallurgy',
                'D': 'Textiles',
                'E': 'Fixed Constructions',
                'F': 'Mechanical Engineering',
                'G': 'Physics',
                'H': 'Electricity',
                'Y': 'Emerging Cross-Sectional Technologies'
               }

In [ ]:
train['cpc'] = train['cate'].apply(lambda x: dict_cpc.get(x, 0))
test['cpc'] = test['cate'].apply(lambda x: dict_cpc.get(x, 0))

In [ ]:
y=train.score
train["input"] = train['anchor'].astype(str)+" "+  train['cpc'].astype(str)+" "+ train['context'].astype(str)+" "+ train['target'].astype(str)
test["input"] = test['anchor'].astype(str) +" "+ test['target'].astype(str)+" "+ test['cpc'].astype(str)+" "+ test['context'].astype(str)

In [ ]:
stop_words = stopwords.words('english')

In [ ]:
def future(df):
    df['anchor_token'] = df['anchor'].apply(lambda x: word_tokenize(x))
    df['anchor_len'] = df['anchor_token'].apply(lambda x: len(x))
    df['target_token'] = df['target'].apply(lambda x: word_tokenize(x))
    df['target_len'] = df['target_token'].apply(lambda x: len(x))
    df['anchor_nostop'] = df['anchor_token'].apply(lambda x: [word for word in x if word not in stop_words])
    df['target_nostop'] = df['target_token'].apply(lambda x: [word for word in x if word not in stop_words])
    return df

In [ ]:
traindf=future(train)
testdf=future(test)

In [ ]:
tr_ts=['input','context','anchor_token','target_token','target_len','target_nostop','anchor_nostop']
y=train.score.values

In [ ]:
idx = traindf.shape[0]
data = pd.concat([traindf, testdf], sort=False)

In [ ]:
data[tr_ts].to_csv('data_sessions_text.txt', 
                                 sep=' ', index=None, header=None)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

cv = TfidfVectorizer(ngram_range=(1,3), max_features=40000)
with open('data_sessions_text.txt') as inp_file:
    data = cv.fit_transform(inp_file)

In [ ]:
X_train = data[:idx]
X_test = data[idx:]
print(X_train.shape, X_test.shape)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y, test_size=0.2, random_state=42)

In [ ]:
reg = Ridge(alpha=0.2, solver='auto')
reg.fit(X_train, y_train)
y_pred = np.expm1(reg.predict(X_valid))
print(pearsonr(y_valid, y_pred)[0])


In [ ]:
import xgboost as xgb
xg_reg = xgb.XGBRegressor( n_estimators=900,booster='gbtree',objective='reg:linear',
       n_jobs=1, random_state=42,importance_type='gain',
       reg_alpha=0.01, reg_lambda=2.9)
xg_reg.fit(X_train, y_train)

y_pred = np.expm1(xg_reg.predict(X_valid))
print(pearsonr(y_valid, y_pred)[0])

In [ ]:
predict = xg_reg.predict(X_test)

In [ ]:
test['score'] = predict
testdf = test[['id', 'score']]
testdf

In [ ]:
testdf.to_csv('submission.csv', index = False)